In [47]:
import typing
from typing import Any, Tuple
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow import keras
 
import numpy as np
import pandas as pd
import io
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [98]:
# file_eng = 'europarl-v7.fr-en.en'
# file_fr = 'europarl-v7.fr-en.fr'

# def load_data(path):
    
#     with  open(path, 'r', encoding="utf8") as f:
#         lines = f.read().splitlines() 
#     return lines[1100:1200]

# clean_eng = load_data(file_eng)
# clean_french = load_data(file_fr)

path = 'eng_-french.csv'

df = pd.read_csv('eng_-french.csv')
df = df.rename(columns={"English words/sentences":"Eng", "French words/sentences":"French" })
 
english = df['Eng']
french = df['French']
clean_eng = english[0:14555]
clean_french = french[0:14555]

targ = list(clean_french)
inp = list(clean_eng)
print(len(clean_eng),len(clean_french))
max_w = 0
m=None
for i in clean_eng:
    if len(i.split(' '))>max_w:
        max_w = len(i.split(' '))
        m = i
print(max_w,m)

14555 14555
5 I like R & B.


In [99]:
dict_inp=set(' '.join(inp).split(' '))
dict_targ=set(' '.join(targ).split(' '))
dict_inp = list(dict_inp)
dict_targ = list(dict_targ)
dict_inp.sort()
dict_targ.sort()
max_len_inp = len(dict_inp)
max_len_targ = len(dict_targ)
print(len(dict_inp),len(dict_targ))
print(max_len_inp,max_len_targ) 

4152 7912
4152 7912


In [100]:
# token_id_inp = dict([(i,token) for i,token in enumerate(dict_inp)])
# token_id_targ = dict([(i,token) for i,token in enumerate(dict_targ)])
# max_len_inp = max([len(line) for line in inp])
# max_len_targ = max([len(line) for line in targ])
# print(max_len_inp)
# print(max_len_targ)

In [101]:
# tokenizer_inp = keras.preprocessing.text.Tokenizer()
# tokenizer_targ = keras.preprocessing.text.Tokenizer()

# tokenizer_inp.fit_on_texts(inp)
# tokenizer_targ.fit_on_texts(targ)

# tokenized_inp = tokenizer_inp.texts_to_sequences(inp) 
# tokenized_targ = tokenizer_targ.texts_to_sequences(targ) 
# padded_inp = keras.preprocessing.sequence.pad_sequences(tokenized_inp,padding='post')
# padded_targ = keras.preprocessing.sequence.pad_sequences(tokenized_targ,padding='post')

In [102]:
preprocess_inp = preprocessing.TextVectorization(max_tokens=max_len_inp) 
preprocess_inp.adapt(inp)
preprocess_targ = preprocessing.TextVectorization(max_tokens=max_len_targ) 
preprocess_targ.adapt(targ)
vocab_inp = np.array(preprocess_inp.get_vocabulary())
vocab_targ = np.array(preprocess_targ.get_vocabulary())
x = preprocess_inp(inp).numpy()
y = preprocess_targ(targ).numpy()

In [103]:
# vocab_inp[preprocess_inp(inp[0]).numpy()]
# preprocess_inp(inp[0])
# len(vocab_inp)
x.shape,y.shape

((14555, 5), (14555, 11))

In [104]:
embedding_inp = keras.layers.Embedding(input_dim=len(vocab_inp),output_dim=128,mask_zero=True)
embedding_targ = keras.layers.Embedding(input_dim=len(vocab_targ),output_dim=128,mask_zero=True)

xx = embedding_inp(x)
yy = embedding_targ(y)
xx.shape,yy.shape
time_inp = xx.shape[1]
time_targ = yy.shape[1]


In [105]:
batch_size = 128
model = keras.Sequential()

model.add(keras.layers.Embedding(len(vocab_inp),batch_size,input_length=time_inp,mask_zero=True))
model.add(keras.layers.LSTM(batch_size)) 
model.add(keras.layers.RepeatVector(time_targ))
model.add(keras.layers.LSTM(batch_size,return_sequences=True))
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(len(vocab_targ),activation='softmax')))

model.compile(loss=keras.losses.sparse_categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 5, 128)            359296    
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 11, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 11, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 11, 128)           0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 11, 6309)          813861    
Total params: 1,436,325
Trainable params: 1,436,325
Non-trainable params: 0
____________________________________________

In [107]:
model.fit(x,y,epochs=10,verbose=2,validation_split=0.1)

Epoch 1/10
410/410 - 19s - loss: 2.1053 - accuracy: 0.7155 - val_loss: 2.3853 - val_accuracy: 0.6682
Epoch 2/10
410/410 - 18s - loss: 1.9848 - accuracy: 0.7212 - val_loss: 2.3349 - val_accuracy: 0.6724
Epoch 3/10
410/410 - 19s - loss: 1.9127 - accuracy: 0.7261 - val_loss: 2.2960 - val_accuracy: 0.6882
Epoch 4/10
410/410 - 19s - loss: 1.8410 - accuracy: 0.7335 - val_loss: 2.2429 - val_accuracy: 0.6974
Epoch 5/10
410/410 - 19s - loss: 1.7675 - accuracy: 0.7402 - val_loss: 2.2153 - val_accuracy: 0.6988
Epoch 6/10
410/410 - 18s - loss: 1.6948 - accuracy: 0.7490 - val_loss: 2.1648 - val_accuracy: 0.7067
Epoch 7/10
410/410 - 19s - loss: 1.6237 - accuracy: 0.7552 - val_loss: 2.1344 - val_accuracy: 0.7197
Epoch 8/10
410/410 - 19s - loss: 1.5659 - accuracy: 0.7607 - val_loss: 2.0926 - val_accuracy: 0.7200
Epoch 9/10
410/410 - 19s - loss: 1.5102 - accuracy: 0.7661 - val_loss: 2.0767 - val_accuracy: 0.7228
Epoch 10/10
410/410 - 21s - loss: 1.4619 - accuracy: 0.7705 - val_loss: 2.0567 - val_accura

In [23]:
x[0].shape

(87,)

In [115]:
test = ['I have a better idea']
tokens = preprocess_inp(test)
# print(tokens)
pred = model.predict(tokens)
vocab_targ[ np.argmax(pred,axis=2)]
# print(pred[0][3])
# pred

array([['jai', 'un', 'un', '', '', '', '', '', '', '', '']], dtype='<U16')